<a href="https://colab.research.google.com/github/AmazeAravind-5904/gemini-q-a-bot-main/blob/main/Gemini_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-generativeai PyMuPDF sentence-transformers scikit-learn faiss-cpu torch nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.3 MB/s eta 0:00:00


In [2]:
import os
import fitz
import google.generativeai as genai
from sentence_transformers import SentenceTransformer, InputExample, losses
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from torch.utils.data import DataLoader
import numpy as np
import textwrap
import nltk
import faiss
import random
from nltk.tokenize import sent_tokenize

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
os.environ["WANDB_DISABLED"] = "true"

In [6]:
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [7]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving LLM.pdf to LLM.pdf


In [8]:
def extract_text(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text
document_text = extract_text(filename)
print("✅ Document loaded successfully.")

✅ Document loaded successfully.


In [9]:
import nltk
nltk.download('punkt_tab')

def split_text(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    curr_chunk = []
    curr_length = 0

    for sentence in sentences:
        token_count = len(sentence.split())
        if curr_length + token_count > max_tokens:
            chunks.append(" ".join(curr_chunk))
            curr_chunk = []
            curr_length = 0
        curr_chunk.append(sentence)
        curr_length += token_count

    if curr_chunk:
        chunks.append(" ".join(curr_chunk))

    return chunks

chunks = split_text(document_text)
print(f"✅ Total chunks created: {len(chunks)}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


✅ Total chunks created: 421


In [10]:
base_model = "paraphrase-MiniLM-L3-v2"
model = SentenceTransformer(base_model)

train_examples = []
for chunk in chunks:
    sentences = sent_tokenize(chunk)
    for i in range(len(sentences) - 1):
        sent1, sent2 = sentences[i], sentences[i + 1]
        # Positive pair (neighboring sentences)
        train_examples.append(InputExample(texts=[sent1, sent2], label=1.0))
        # Negative pair (random sentence)
        neg = random.choice(sentences)
        train_examples.append(InputExample(texts=[sent1, neg], label=0.0))

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

print("⏳ Fine-tuning SBERT...")
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=2,
    warmup_steps=100,
    show_progress_bar=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


⏳ Fine-tuning SBERT...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.278700
1000,0.266300
1500,0.258600
2000,0.256100


In [11]:
model.save("fine_tuned_sbert")
print("✅ Fine-tuned SBERT model saved.")

✅ Fine-tuned SBERT model saved.


In [12]:
model = SentenceTransformer("fine_tuned_sbert")
embeddings = model.encode(chunks, show_progress_bar=True).astype("float32")

dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)
print(f"✅ FAISS index created with {faiss_index.ntotal} vectors.")


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

✅ FAISS index created with 421 vectors.


In [13]:
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = faiss_index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

In [14]:
gen_model = genai.GenerativeModel(model_name="gemini-2.0-flash")

def ask_gemini(question):
    relevant_chunks = retrieve_relevant_chunks(question)
    context = "\n\n".join(relevant_chunks)

    prompt = f"""Use the following document information to answer the question.
    Context: {context}
    Question: {question}
    Answer:"""

    response = gen_model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(temperature=0.2)
    )
    return str(response.text).strip()

In [21]:
sample_questions = ["What is Instruction Fine-tuning?"]

for question in sample_questions:
    print("\nQuestion:", question)
    answer = ask_gemini(question)
    print("Answer:\n", textwrap.fill(answer, width=80))


Question: What is Instruction Fine-tuning?
Answer:
 Instruction fine-tuning is a method where a model's parameters are slightly
adjusted using instruction-following data to activate the general language
knowledge acquired during pre-training and apply it to new tasks. It involves
fine-tuning the model on a dataset of instruction-response pairs, where the
input includes instructions, system information, and user-provided information,
and the output is the desired response.
